Instructions to run this utility:

1. Define the path to the three CSVs in the following variables:
    a. filepath_sales, filepath_prod_hierarchy and filepath_store_cities
2. Define the path to the pickle files shared in the submission folder in the following variables:
    a. filepath_outliers (model_outliers_isolation_forest.pkl), filepath_scaler (model_scaler.pkl), filepath_pca (model_pca_post_knee_plot.pkl), filepath_predictor (model_pca_post_knee_plot.pkl)
3. Run the below code snippets in the sequence as defined below.
4. Mean Squared Error would be displayed at the end of this notebook.

PS: In case of any errors, restart and clear the output and run the code again.

In [1]:
import numpy as np                     
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor
import pickle
from datetime import date

In [2]:
%matplotlib inline
sns.set(style='whitegrid', font_scale=1.3, color_codes=True)      # To apply seaborn styles to the plots.
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [10]:
filepath_sales = "sales.csv"
filepath_prod_hierarchy = "product_hierarchy.csv"
filepath_store_cities = "store_cities.csv"

In [11]:
def load_data(filepath_sales, filepath_prod_hierarchy, filepath_store_cities):
    df_sales = pd.read_csv(filepath_sales, nrows=1000)
    df_prod_hierarchy = pd.read_csv(filepath_prod_hierarchy, nrows=1000)
    df_store_cities = pd.read_csv(filepath_store_cities, nrows=1000)
    return df_sales, df_prod_hierarchy, df_store_cities
df_sales, df_prod_hierarchy, df_store_cities = load_data(filepath_sales, filepath_prod_hierarchy, filepath_store_cities)

In [12]:
def convert_date_to_date(df_sales):
    df_sales['date'] = pd.to_datetime(df_sales['date'], infer_datetime_format=True)
    df_sales['weekday'] = df_sales.date.dt.weekday 
    df_sales['month'] = df_sales.date.dt.month
    epoch= pd.Timestamp(df_sales.date.min())  
    df_sales["week_number_c"]=np.where(df_sales.date.astype("datetime64").le(epoch), \
                               df_sales.date.dt.isocalendar().week, \
                               df_sales.date.sub(epoch).dt.days//7).astype("int64") #+52)
    return df_sales
df_sales = convert_date_to_date(df_sales)

In [13]:
def fill_null_values(df):
    df.promo_bin_1.fillna("None", inplace=True)
    df.promo_bin_2.fillna("None", inplace=True)
    df.promo_discount_type_2.fillna("None", inplace=True)
    df['sales'] = df['sales'].fillna(0)
    df['revenue'] = df['revenue'].fillna(0)
    df['price'] = df['price'].fillna(df['revenue']/df['sales'])
    df['stock'] = df['stock'].fillna(0)
    df['price'] = df['price'].fillna(0)
    if 'sales'==0 in df.columns:
        df['promo_discount_2'] == 0
    else:
        df['promo_discount_2'] = (((df['price']*df['sales'])-df['revenue'])/(df['price']*df['sales']))*100
    df['promo_discount_2'] = df['promo_discount_2'].fillna(0)
    return df

df_sales = fill_null_values(df_sales)

In [14]:
def merge_data(df_sales, df_prod_hierarchy, df_stores_cities):
    df_sales_new = df_sales.drop(['promo_bin_1','promo_bin_2','promo_discount_type_2'], axis=1)

    df_product_new = df_prod_hierarchy.drop(['product_length','product_depth','product_width'], axis=1)

    sales_product_df = pd.merge(df_sales_new,df_product_new, on='product_id',how='left')

    master_df = pd.merge(sales_product_df, df_stores_cities, on='store_id', how='left')
    return master_df

df_merged_data = merge_data(df_sales, df_prod_hierarchy, df_store_cities)

In [15]:
def sales_grouped_data(df):
    grouped_data = df.groupby(
       ['city_id', 'store_id','product_id','week_number_c']
    ).agg(
        {
            'sales':sum,    # Sum of sales
            'revenue': sum,  # Sum of revenue
            'stock': sum,
            'price': 'mean'
        }
    )
    grouped_data.reset_index(inplace=True)
    return grouped_data

grouped_data_final = sales_grouped_data(df_merged_data)

In [16]:
type(grouped_data_final)

pandas.core.frame.DataFrame

In [17]:
grouped_data_final

,city_id,store_id,product_id,week_number_c,sales,revenue,stock,price
0,C002,S0104,P0001,1,0.00,0.00,22.00,6.25
1,C002,S0104,P0005,1,0.00,0.00,3.00,33.90
2,C002,S0104,P0011,1,0.00,0.00,8.00,49.90
3,C002,S0104,P0015,1,0.00,0.00,11.00,2.60
4,C002,S0104,P0017,1,0.00,0.00,83.00,1.49
5,C002,S0104,P0018,1,0.00,0.00,78.00,1.95
6,C002,S0104,P0024,1,0.00,0.00,28.00,1.95
7,C002,S0104,P0035,1,1.00,2.27,33.00,2.45
8,C002,S0104,P0046,1,0.00,0.00,9.00,34.50
9,C002,S0104,P0051,1,4.00,2.59,274.00,0.70


In [18]:
def data_prep(grouped_data_final):
    grouped_data_final['ID'] = grouped_data_final['product_id'] + grouped_data_final['store_id']
    grouped_data_final.drop(['product_id', 'store_id'], axis=1, inplace=True)
    grouped_data_final.set_index(['ID'], inplace=True)
    return grouped_data_final
grouped_data_final = data_prep(grouped_data_final)

In [19]:
grouped_data_final

,city_id,week_number_c,sales,revenue,stock,price
ID,,,,,,
P0001S0104,C002,1,0.00,0.00,22.00,6.25
P0005S0104,C002,1,0.00,0.00,3.00,33.90
P0011S0104,C002,1,0.00,0.00,8.00,49.90
P0015S0104,C002,1,0.00,0.00,11.00,2.60
P0017S0104,C002,1,0.00,0.00,83.00,1.49
P0018S0104,C002,1,0.00,0.00,78.00,1.95
P0024S0104,C002,1,0.00,0.00,28.00,1.95
P0035S0104,C002,1,1.00,2.27,33.00,2.45
P0046S0104,C002,1,0.00,0.00,9.00,34.50


In [20]:
def encode_cat_features(grouped_data_final):
    df_with_dummies = pd.get_dummies(grouped_data_final,columns=grouped_data_final.select_dtypes(include=['object']).columns, drop_first=True)
    return df_with_dummies
df_with_dummies = encode_cat_features(grouped_data_final)

In [21]:
# Since the test data had less number of features, so dummy features are added    
if len(df_with_dummies.columns) < 41:
    for i in range(42 - len(df_with_dummies.columns)):
        df_with_dummies['city_id_C' + str(i)] = np.nan
    df_with_dummies = df_with_dummies.fillna(0)
elif len(df_with_dummies.columns) > 41:
    print('Incorrect No of Features')    

In [22]:
label = 'sales'

In [23]:
def get_test_data(df_with_dummies):
    X_test, y_test = df_with_dummies.drop([label], axis=1).values, df_with_dummies[label].values
    return X_test, y_test
X_test, y_test = get_test_data(df_with_dummies)

In [29]:
filepath_outliers = "./Pickles/model_outliers_isolation_forest.pkl"
filepath_scaler = "./Pickles/model_scaler.pkl"
filepath_pca = "./Pickles/model_pca_post_knee_plot.pkl"
filepath_predictor = "./Pickles/model_xgboost_regressor_tree.pkl"

def load_pickle_files(filepath_outliers, filepath_scaler, filepath_pca, filepath_predictor):
    objects = []
    with (open(filepath_outliers, "rb")) as model_outliers_isolation_forest:
        while True:
            try:
                objects.append(pickle.load(model_outliers_isolation_forest))
            except EOFError:
                break
    with (open(filepath_scaler, "rb")) as model_scaler:
        while True:
            try:
                objects.append(pickle.load(model_scaler))
            except EOFError:
                break
    with (open(filepath_pca, "rb")) as model_pca_post_knee_plot:
        while True:
            try:
                objects.append(pickle.load(model_pca_post_knee_plot))
            except EOFError:
                break
    with (open(filepath_predictor, "rb")) as model_xgboost_regressor_tree:
        while True:
            try:
                objects.append(pickle.load(model_xgboost_regressor_tree))
            except EOFError:
                break
    return objects
models = load_pickle_files(filepath_outliers, filepath_scaler, filepath_pca, filepath_predictor)

In [30]:
models

[IsolationForest(random_state=3),
 StandardScaler(),
 PCA(n_components=39, random_state=0),
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, nthread=2, num_parallel_tree=1,
              random_state=3, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)]

In [31]:
def handle_outliers(X_test, y_test):
    outlier_preds_test = models[0].predict(X_test)
    mask = outlier_preds_test != -1
    X_test_wo_outliers = X_test[mask, :]
    y_test_wo_outliers = y_test[mask]
    return X_test_wo_outliers, y_test_wo_outliers
X_test_wo_outliers, y_test_wo_outliers = X_test, y_test

In [32]:
def scale_data(X_test_wo_outliers):
#     model_scaler = StandardScaler(with_mean=False)
#     X_test_wo_outliers_scaled = model_scaler.fit_transform(X_test_wo_outliers)
    X_test_wo_outliers_scaled = models[1].fit_transform(X_test_wo_outliers)
    return X_test_wo_outliers_scaled
X_test_wo_outliers_scaled = scale_data(X_test_wo_outliers)

In [33]:
def feature_selection_pca(X_test_wo_outliers_scaled):
    X_test_wo_outliers_scaled_post_PCA = models[2].transform(X_test_wo_outliers_scaled)
    return X_test_wo_outliers_scaled_post_PCA
X_test_wo_outliers_scaled_post_PCA = feature_selection_pca(X_test_wo_outliers_scaled)

In [34]:
X_test_wo_outliers_scaled_post_PCA

array([[-0.35236684, -0.18778826, -1.32912574, ..., -0.09199853,
         1.43326106, -0.4130043 ],
       [ 0.07190866, -0.19283624, -1.95203138, ..., -0.65501149,
        -0.19329733, -1.22192216],
       [ 0.46908341, -0.18440275, -2.28998303, ..., -0.96828868,
        -1.11103932, -1.68200465],
       ...,
       [-0.28488892,  0.19957847, -1.25070221, ..., -1.07224882,
         0.99895262,  0.02541851],
       [-0.50925659,  0.17615659, -1.33886615, ..., -1.14502965,
         0.79628019, -0.07908818],
       [-0.48393075,  0.16553781, -1.56827292, ..., -1.3462477 ,
         0.22489531, -0.36036928]])

In [35]:
def predict(X_test_wo_outliers_scaled_post_PCA, y_test_wo_outliers):
    y_pred = models[3].predict(X_test_wo_outliers_scaled_post_PCA)
    return y_pred
y_pred = predict(X_test_wo_outliers_scaled_post_PCA, y_test_wo_outliers)

In [36]:
def calculate_score(y_pred, y_test_wo_outliers):
    return mean_squared_error(y_pred, y_test_wo_outliers)
model_mse = calculate_score(y_pred, y_test_wo_outliers)